# **Control SDP with the `ska-sdp` CLI**

The `ska-sdp` command line interface can be used to directly interact with Science Data Processor's (SDP)
Configuration Database (DB). The Config DB (or DB) contains information about the tango devices,
available processing scripts, and processes currently running in SDP. It is the heart of the system,
all the sub-systems connect to it in order to know what processes need running, which ones have stopped
(successfully or with an error), etc.

The code of the CLI is stored in the [SDP Configuration Library](https://gitlab.com/ska-telescope/sdp/ska-sdp-config/-/tree/master/src/ska_sdp_config/ska_sdp_cli) GitLab repository.

What you will learn from this notebook is:
 - how to learn to use ska-sdp (i.e. how to use its help function)
 - how to list and inspect entries in the Config DB
 - how run a processing script (both batch and realtime)
 - how to create and delete entries from the Config DB
 - how to import your unpublished, custom processing script so that you can run it in SDP
 
 If you get stuck you can look at the official documentation pages:
 - [SDP Integration](https://developer.skao.int/projects/ska-sdp-integration/en/latest/index.html)
 - [ska-sdp CLI](https://developer.skao.int/projects/ska-sdp-config/en/latest/cli.html)
 - [SDP Processing Scripts](https://developer.skao.int/projects/ska-sdp-script/en/latest/) 
 - [SDP on the Developer Portal](https://developer.skao.int/en/latest/projects/area/sdp.html)
 
 If you still don't find the answer to your questions, contact us on Slack: #......

In [ ]:
import os
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.<update-with-ns!!!>"

## 1. ska-sdp CLI `help` function

In [3]:
!ska-sdp --help

Command line utility for interacting with SKA Science Data Processor (SDP).

Usage:
    ska-sdp COMMAND [options] [SDP_OBJECT] [<args>...]
    ska-sdp COMMAND (-h|--help)
    ska-sdp (-h|--help)

SDP Objects:
    pb           Interact with processing blocks
    script       Interact with available processing script definitions
    deployment   Interact with deployments
    eb           Interact with execution blocks
    controller   Interact with Tango controller device
    subarray     Interact with Tango subarray device

Commands:
    list           List information of object from the Configuration DB
    get | watch    Print all the information (i.e. value) of a key in the Config DB
    create         Create a new, raw key-value pair in the Config DB;
                   Run a processing script; Create a deployment
    update         Update a raw key value from CLI
    edit           Edit a raw key value from text editor
    delete         Delete a single key or all keys within a pat

#### Each sub-command has their own help. Look at `ska-sdp create` for example:

In [4]:
!ska-sdp create --help

Create SDP objects (deployment, script, eb) in the Configuration Database.
Create a processing block to run a script.

Usage:
    ska-sdp create [options] pb <script> [<parameters>] [--eb=<eb-parameters>]
    ska-sdp create [options] deployment <item-id> <kind> <parameters>
    ska-sdp create [options] (script|eb) <item-id> <value>
    ska-sdp create (-h|--help)

Arguments:
    <script>            Script that the processing block will run, in the format:
                            kind:name:version
    <parameters>        Optional parameters for a script, with expected format:
                            '{"key1": "value1", "key2": "value2"}'
                        For deployments, expected format:
                            '{"chart": <chart-name>, "values": <dict-of-values>}'
    <eb-parameters>     Optional eb parameters for a real-time script
    <item-id>           Id of the new deployment, script or eb
    <kind>              Kind of the new deployment (currently "helm" only)


## 2. `list`: look at the content of the Configuration Database

In [6]:
!ska-sdp list -a

Keys with prefix /: 
/lmc/controller
/lmc/subarray/01
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


In the above output, the default contents of the DB are shown:
    - the tango devices: controller (/lmc/controller) and subarray(s) (/lmc/subarray/01)
    - processing script definitions (entries starting with /script/)
    
As you run scripts on SDP, more entries will appear, including processing blocks (pb) and execution blocks (eb).

## 3. `get`: look at the content of an entry

In [7]:
!ska-sdp get /lmc/controller

/lmc/controller = {
  "state": "STANDBY",
  "transaction_id": null
}


In [8]:
!ska-sdp get /script/realtime:pss-receive:0.3.0

/script/realtime:pss-receive:0.3.0 = {
  "image": "artefact.skao.int/ska-sdp-script-pss-receive:0.3.0"
}


Note: if you already have processing blocks (pb) in the DB, you can get the content of all of the ones that belong to the same processing by passing the pb id to the command:

    ska-sdp get pb <pb_id>

We will come back to this when there will be relevant entries in the DB

## 4. Starting a `batch` processing script

In order to start a processing script, we need to `create` a processing block. Let's take a look at the `ska-sdp create` command's help again:

In [9]:
!ska-sdp create --help

Create SDP objects (deployment, script, eb) in the Configuration Database.
Create a processing block to run a script.

Usage:
    ska-sdp create [options] pb <script> [<parameters>] [--eb=<eb-parameters>]
    ska-sdp create [options] deployment <item-id> <kind> <parameters>
    ska-sdp create [options] (script|eb) <item-id> <value>
    ska-sdp create (-h|--help)

Arguments:
    <script>            Script that the processing block will run, in the format:
                            kind:name:version
    <parameters>        Optional parameters for a script, with expected format:
                            '{"key1": "value1", "key2": "value2"}'
                        For deployments, expected format:
                            '{"chart": <chart-name>, "values": <dict-of-values>}'
    <eb-parameters>     Optional eb parameters for a real-time script
    <item-id>           Id of the new deployment, script or eb
    <kind>              Kind of the new deployment (currently "helm" only)


According to this, we need to specify the script we want in the form of `kind:name:version`. We can provide our script with its expected parameters via the `parameters` argument in dictionary format.

We saw above which processing scripts are available in the Config DB. Only these can be executed by SDP. If you want a different one, you will have to add its details first to the DB (see later).

Let's start the [test_dask script](https://developer.skao.int/projects/ska-sdp-script/en/latest/test-scripts/test-dask.html)

In [15]:
!ska-sdp create pb batch:test-dask:0.3.0

Processing block created with pb_id: pb-sdpcli-20220729-00001


Let's list the contents of the DB again.

In [16]:
!ska-sdp list -a

Keys with prefix /: 
/deploy/proc-pb-sdpcli-20220729-00000-script
/deploy/proc-pb-sdpcli-20220729-00001-script
/lmc/controller
/lmc/subarray/01
/pb/pb-sdpcli-20220729-00000
/pb/pb-sdpcli-20220729-00000/state
/pb/pb-sdpcli-20220729-00001
/pb/pb-sdpcli-20220729-00001/state
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


We see six new entries in the database:
 - three starting with /deploy/
 - three starting with /pb/
 
The deploy entries contain informations about the deployments that SDP started and the pb entries contain the processing block information. If the processing block is deleted from the DB, the deployment connected to it is also deleted automatically.

Let's inspect these entries in more detail.

In [17]:
# First let's inspect all of the processing blocks at once, with `get pb`
# remember to use the correct pb id from the DB
!ska-sdp get pb pb-sdpcli-20220729-00000

/pb/pb-sdpcli-20220729-00001 = {
  "dependencies": [],
  "eb_id": null,
  "parameters": {},
  "pb_id": "pb-sdpcli-20220729-00001",
  "script": {
    "kind": "batch",
    "name": "test-dask",
    "version": "0.3.0"
  }
}
/pb/pb-sdpcli-20220729-00001/owner = {
  "command": [
    "test_dask.py"
  ],
  "hostname": "proc-pb-sdpcli-20220729-00001-script--1-2dbvj",
  "pid": 1
}
/pb/pb-sdpcli-20220729-00001/state = {
  "deployments": {
    "proc-pb-sdpcli-20220729-00001-dask-1": "RUNNING",
    "proc-pb-sdpcli-20220729-00001-dask-2": "RUNNING"
  },
  "resources_available": true,
  "status": "RUNNING"
}


Depending on how fast we executed the above command, we may see that the processing has already finished. We can learn about which script was executed, what deployments it started, and what the status of the processing is. Once the deployment finishes, only the deploy object of the processing script remains. If the script deployed other objects, like in our case the Dask execution engine, then those will be deleted upon finishing their processing.

In [18]:
!ska-sdp get /deploy/proc-pb-sdpcli-20220729-00000-script

/deploy/proc-pb-sdpcli-20220729-00001-script = {
  "args": {
    "chart": "script",
    "values": {
      "env": [
        {
          "name": "SDP_CONFIG_HOST",
          "value": "ska-sdp-etcd-client.dp-orca-gabi"
        },
        {
          "name": "SDP_HELM_NAMESPACE",
          "value": "dp-orca-gabi-p"
        },
        {
          "name": "SDP_PB_ID",
          "value": "pb-sdpcli-20220729-00001"
        }
      ],
      "image": "artefact.skao.int/ska-sdp-script-test-dask:0.3.0"
    }
  },
  "dpl_id": "proc-pb-sdpcli-20220729-00001-script",
  "kind": "helm"
}


Above, we see the contents of the deployment of the actual processing script. It tells you whit what parameters (values) and which chart was deployed.

Here is an alternative way of getting the details of all objects of the same type using `list`:

In [19]:
!ska-sdp list -v deployment

Keys with prefix /deploy: 
/deploy/proc-pb-sdpcli-20220729-00000-script = {
  "args": {
    "chart": "script",
    "values": {
      "env": [
        {
          "name": "SDP_CONFIG_HOST",
          "value": "ska-sdp-etcd-client.dp-orca-gabi"
        },
        {
          "name": "SDP_HELM_NAMESPACE",
          "value": "dp-orca-gabi-p"
        },
        {
          "name": "SDP_PB_ID",
          "value": "pb-sdpcli-20220729-00000"
        }
      ],
      "image": "artefact.skao.int/ska-sdp-script-test-dask:0.3.0"
    }
  },
  "dpl_id": "proc-pb-sdpcli-20220729-00000-script",
  "kind": "helm"
}
/deploy/proc-pb-sdpcli-20220729-00001-dask-1 = {
  "args": {
    "chart": "dask",
    "values": {
      "image": "artefact.skao.int/ska-sdp-script-test-dask:0.3.0",
      "worker.replicas": 2
    }
  },
  "dpl_id": "proc-pb-sdpcli-20220729-00001-dask-1",
  "kind": "helm"
}
/deploy/proc-pb-sdpcli-20220729-00001-dask-2 = {
  "args": {
    "chart": "dask",
    "values": {
      "image": "artefac

Now let's clean up after ourselves with `ska-sdp delete`

In [22]:
!ska-sdp delete pb pb-sdpcli-20220729-00000

/pb/pb-sdpcli-20220729-00001
/pb/pb-sdpcli-20220729-00001/state
Deleted above keys with prefix /pb/pb-sdpcli-20220729-00001.


Once we delete the processing block, its deployment is also deleted:

In [24]:
!ska-sdp list -a

Keys with prefix /: 
/lmc/controller
/lmc/subarray/01
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


## 5. Starting a `real-time` processing script

Running real-time scripts requires more information. Apart from proving the necessary parameters (in the form of a json string), you need to specific an execution block. Real time scripts are only executed if they are part of a schedule, which are defined by execution blocks. We will use the [test-realtime processing script](https://developer.skao.int/projects/ska-sdp-script/en/latest/test-scripts/test-realtime.html) for demonstration. But if you are interested in running the visibility receive one, follow the instructions on the [script's documentation page](https://developer.skao.int/projects/ska-sdp-script/en/latest/scripts/vis-receive.html), and see how you can run it with the ska-sdp [CLI](https://developer.skao.int/projects/ska-sdp-script/en/latest/scripts/vis-receive.html#cli-bash). 

In [26]:
!ska-sdp create pb realtime:test-realtime:0.3.0 --eb="{\
  'scan_types': [\
    {\
      'id': 'science_A',\
      'coordinate_system': 'ICRS', 'ra': '02:42:40.771', 'dec': '-00:00:47.84',\
      'channels': [\
        { 'count': 5, 'start': 0, 'stride': 2, 'freq_min': 0.35e9, 'freq_max': 0.368e9, 'link_map': [[0,0], [200,1], [744,2], [944,3]] }\
      ]\
    },\
    {\
      'id': 'calibration_B',\
      'coordinate_system': 'ICRS', 'ra': '12:29:06.699', 'dec': '02:03:08.598',\
      'channels': [\
        { 'count': 5, 'start': 0, 'stride': 2, 'freq_min': 0.35e9, 'freq_max': 0.368e9, 'link_map': [[0,0], [200,1], [744,2], [944,3]] }\
      ]\
    }\
  ]\
}"

Execution block created with eb_id: eb-sdpcli-20220729-00000
The EB can be ended by running: ska-sdp end eb eb-sdpcli-20220729-00000
Processing block created with pb_id: pb-sdpcli-20220729-00000


In [27]:
# Let's take a look at the entries in the DB
!ska-sdp list -a

Keys with prefix /: 
/deploy/proc-pb-sdpcli-20220729-00000-script
/eb/eb-sdpcli-20220729-00000
/lmc/controller
/lmc/subarray/01
/pb/pb-sdpcli-20220729-00000
/pb/pb-sdpcli-20220729-00000/owner
/pb/pb-sdpcli-20220729-00000/state
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


Now we have a deploymen (/deploy), a set of processing blocks (/pb), and an execution block (/eb). The processing script will run as long as the execution block is running. We need to end it in order to end the processing.

In [33]:
# First, let's check that the script is runing 
!ska-sdp get /pb/pb-sdpcli-20220729-00000/state

/pb/pb-sdpcli-20220729-00000/state = {
  "deployments": {},
  "resources_available": true,
  "status": "RUNNING"
}


In [34]:
# Now let's end the eb; you may copy the command in the output of the create command
!ska-sdp end eb eb-sdpcli-20220729-00000

EB eb-sdpcli-20220729-00000 FINISHED 


In [36]:
# Now the state of the pb is "FINISHED"
!ska-sdp get /pb/pb-sdpcli-20220729-00000/state

/pb/pb-sdpcli-20220729-00000/state = {
  "deployments": {},
  "resources_available": true,
  "status": "FINISHED"
}


In [37]:
!ska-sdp list -a

Keys with prefix /: 
/deploy/proc-pb-sdpcli-20220729-00000-script
/eb/eb-sdpcli-20220729-00000
/lmc/controller
/lmc/subarray/01
/pb/pb-sdpcli-20220729-00000
/pb/pb-sdpcli-20220729-00000/state
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


You can see that the entries are still in the Config DB, so let's clean up.

In [38]:
# First delete the pb and with that the deployment
!ska-sdp delete pb pb-sdpcli-20220729-00000

/pb/pb-sdpcli-20220729-00000
/pb/pb-sdpcli-20220729-00000/state
Deleted above keys with prefix /pb/pb-sdpcli-20220729-00000.


In [39]:
# Then delete the execution block
!ska-sdp delete eb eb-sdpcli-20220729-00000

/eb/eb-sdpcli-20220729-00000
Deleted above keys with prefix /eb/eb-sdpcli-20220729-00000.


## 6. `Import` or `create` processing script definitions

You are welcome to build your own processing scripts (instructions on how you can do that can be found in the [Processing Scripts documentation](https://developer.skao.int/projects/ska-sdp-script/en/latest/script-development.html)). Once you have the docker image present (ideally in an online repository, like GitLab provides for every project), you can run that in SDP once you added some details to the Configuration DB. This is what the `ska-sdp import` command is used for.

In [40]:
!ska-sdp import --help

Import processing script definitions into the Configuration Database.

Usage:
    ska-sdp import scripts [options] <file-or-url>
    ska-sdp import (-h|--help)

Arguments:
    <file-or-url>      File or URL to import script definitions from.

Options:
    -h, --help          Show this screen
    --sync              Delete scripts not in the input


According to the help, you can import scripts from a file or from a URL. There are examples of what structure the files need to contain at the [CLI documentation](https://developer.skao.int/projects/ska-sdp-config/en/latest/cli.html#example-script-definitions-file-content-for-import). 

A simpler way of adding information, especially if you only need to do it for one script definition, is to use the `ska-sdp create` command.

    ska-sdp create [options] (script|eb) <item-id> <value>
    
where `item-id` needs to have the format of <kind>:<name>:<version>, e.g. `realtime:test-script:0.1.0`
and `value` is a json string in the form of `'{"image": "<url-to-image>/test-script:0.1.0"}'`

Let's create one for a fake docker image (we won't be able to run with SDP).

In [42]:
!ska-sdp create script batch:my-test-image:0.0.1 '{"image":random-image-url:0.0.1}'

/script/batch:my-test-image:0.0.1 created


In [43]:
# now let's see that is in the DB and its value is what we specified
!ska-sdp get /script/batch:my-test-image:0.0.1

/script/batch:my-test-image:0.0.1 = {"image":random-image-url:0.0.1}


Now SDP will be able to discover the script and execute it.